# Estrutura

### Painel
<ol>
    <li>Conexão com o painel</li>
    <li>Leitura dos dados</li>
    <li>Formatação e limpeza dos dados</li>
</ol>

### Banco de dados
<ol>
    <li>Conexão com o banco de dados</li>
    <li>Gravação dos dados</li>
</ol>

### Envio para nuvem
<ol>
    <li>Conexão com o banco de dados</li>
    <li>Recuperar os dados</li>
    <li>Formatação dos dados</li>
    <li>Encaminhamento para nuvem</li>
</ol>

### Estrutura de nuvem
<ol>
    <li>Criar nuvem</li>
    <li>Iniciar nuvem</li>
    <li>Recuperar os dados da nuvem</li>
    <li>Plotar</li>
</ol>

In [8]:
# imports
import requests
import psycopg2
import random
import time
import pandas as pd
from datetime import datetime
import streamlit as st
from bs4 import BeautifulSoup

## Painel

In [9]:
# Retorna a feature hora e a tabela do site
def conecta_painel():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"
    }

    url = "https://www.horariodebrasilia.org/"
    url2 = "https://www.canalrural.com.br/cotacao/soja/"

    hora_site = requests.get(url, headers=headers)
    soup_hora = BeautifulSoup(hora_site.content, "html.parser")
    tempo = soup_hora.find("p", id="relogio")

    preco_site = requests.get(url2, headers=headers)
    soup_preco = BeautifulSoup(preco_site.content, "html.parser")

    tabela = soup_preco.find("tbody")

    # tempo = time.strftime("%H:%M:%S", datetime.now())
    tempo = datetime.now()
    # date_str = str(datetime.now())
    # tempo = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S.%f")

    return tempo, tabela


In [10]:
# Retorna a tabela do site como dataframe
def le_dados_painel(hora, tbody):
    colunas = list()
    linhas = list()

    for tr in tbody.find_all("tr"):
        td = tr.find_all("td")[0].text
        colunas.append(td)

    for tr in tbody.find_all("tr"):
        td = tr.find_all("td")[1].text
        td = float(td) + random.randint(-50, 50)
        linhas.append(td)

    colunas.append("CLP")
    linhas.append("CLP_" + str(random.randint(1, 3)))

    linhas = [linhas]

    dataframe = pd.DataFrame(linhas, columns=colunas, index=[hora])

    return dataframe


In [11]:
def formata_dados_painel():
    
    a, b = conecta_painel()
    df = le_dados_painel(a, b)   

    # for i in range(3):
    #     time.sleep(1)
    #     a, b = conecta_painel()
    #     df2 = le_dados_painel(a, b)
    #     df = pd.concat([df, df2])
    # display(df.head())
    # display(df.iloc[0,8])
    return df

formata_dados_painel()

,Dourados (MS),Bebedouro (SP),Balsas (MA),Rondonópólis (MT),Passo Fundo (RS),Pedro Afonso (TO),Cascavel (PR),Rio Verde (GO),CLP
2023-03-24 19:00:39.205825,101.0,118.0,102.0,155.0,124.5,159.0,162.0,182.0,CLP_2


## Banco de dados

In [12]:
postgres=# create database ats;
postgres=# create user almir with encrypted password 'almir';
postgres=# grant all privileges on database ats to almir;
postgres=# alter database ats owner to almir;




import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='postgres', password='password', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Preparing query to create a database
sql = '''CREATE database mydb''';

#Creating a database
cursor.execute(sql)
print("Database created successfully........")

#Closing the connection
conn.close()


SyntaxError: invalid syntax (1835391987.py, line 1)

In [ ]:
def create_database(args):
    pass

In [13]:
def db_connection():
    con = psycopg2.connect(
        host="localhost", 
        database="ats", 
        user="almir", 
        password="almir"
    )
    return con

In [14]:
def create_table():
    # bravo timestamp without time zone DEFAULT now(),
    # Data VARCHAR(255) NOT NULL,

    create = """
        CREATE TABLE supervisory (
            id SERIAL PRIMARY KEY,
            Data VARCHAR(255) NOT NULL,
            Caldeira NUMERIC NOT NULL,
            Pneumatico DECIMAL NOT NULL,
            Rotacao DECIMAL NOT NULL,
            Aquecimento DECIMAL NOT NULL,
            Resfriamento DECIMAL NOT NULL,
            Pressao DECIMAL NOT NULL,
            Umidade DECIMAL NOT NULL,
            Velocidade DECIMAL NOT NULL,
            CLP VARCHAR(255) NOT NULL
        )
        """
    
    con = None
    try:
        con = db_connection()
        cursor = con.cursor()
        cursor.execute("DROP TABLE IF EXISTS supervisory")
        cursor.execute(create)
        # cursor.execute(sql)
        cursor.close()
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if con is not None:
            con.close()

create_table()

In [48]:
def insert_db(df):
    con = None
    try:
        con = db_connection()
        # Setting auto commit false
        con.autocommit = True
        cursor = con.cursor()

        # df = formata_dados_painel()
        
        # Preparing SQL queries to INSERT a record into the database.
        cursor.execute(
            f"""
        INSERT INTO supervisory(            
            Data,
            Caldeira,
            Pneumatico,
            Rotacao,
            Aquecimento,
            Resfriamento,
            Pressao,
            Umidade,
            Velocidade,
            CLP)
            VALUES(
                \'{df.index.values[0]}\',
                {df.iloc[0,0]},
                {df.iloc[0,1]},
                {df.iloc[0,2]},
                {df.iloc[0,3]},
                {df.iloc[0,4]},
                {df.iloc[0,5]},
                {df.iloc[0,6]},
                {df.iloc[0,7]},
                \'{df.iloc[0,8]}\')
        """
        )

        # Commit your changes in the database
        con.commit()
        print("Records inserted........")

        # Closing the connection
        con.close()
        cursor.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if con is not None:
            con.close()


insert_db()


Records inserted........


In [51]:
def get_data_db():
    con = None
    try:
        con = db_connection()
        # Setting auto commit false
        con.autocommit = True
        cursor = con.cursor()

        # Retrieving data
        cursor.execute( '''SELECT * from supervisory''' )

        #Fetching all rows from the table
        result = cursor.fetchall();
        print(result)
        column_names = [desc[0] for desc in cursor.description]
        print(column_names)

        # Commit your changes in the database
        con.commit()

        # Closing the connection
        con.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if con is not None:
            con.close()
    
    return result, column_names

get_data_db()

[]
['id', 'data', 'caldeira', 'pneumatico', 'rotacao', 'aquecimento', 'resfriamento', 'pressao', 'umidade', 'velocidade', 'clp']


([],
 ['id',
  'data',
  'caldeira',
  'pneumatico',
  'rotacao',
  'aquecimento',
  'resfriamento',
  'pressao',
  'umidade',
  'velocidade',
  'clp'])

## Nuvem

In [ ]:
# Criar páginas streamlit


# Barra lateral
with st.sidebar:
    st.title("Controle supervisório")
    st.text(
        """
        Este projeto é um robô desenvolvido 
        em Python para operar Long & Short 
        com ações de empresas brasileiras,
        também conhecido como Pairs Trading
        ou Arbitragem estatística, 
        utilizando cointegração e 
        estacionariedade.
        """
    )


In [2]:
import plotly.express as px

df = px.data.gapminder().query("continent=='Oceania'")
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
60,Australia,Oceania,1952,69.120,8691212,10039.59564,AUS,36
61,Australia,Oceania,1957,70.330,9712569,10949.64959,AUS,36
62,Australia,Oceania,1962,70.930,10794968,12217.22686,AUS,36
63,Australia,Oceania,1967,71.100,11872264,14526.12465,AUS,36
64,Australia,Oceania,1972,71.930,13177000,16788.62948,AUS,36
65,Australia,Oceania,1977,73.490,14074100,18334.19751,AUS,36
66,Australia,Oceania,1982,74.740,15184200,19477.00928,AUS,36
67,Australia,Oceania,1987,76.320,16257249,21888.88903,AUS,36
68,Australia,Oceania,1992,77.560,17481977,23424.76683,AUS,36
69,Australia,Oceania,1997,78.830,18565243,26997.93657,AUS,36


In [1]:
millis=input("Enter time in milliseconds ")
millis = int(millis)
seconds=(millis/1000)%60
seconds = int(seconds)
minutes=(millis/(1000*60))%60
minutes = int(minutes)
hours=(millis/(1000*60*60))%24

print ("%d:%d:%d" % (hours, minutes, seconds))

0:7:33


In [1]:
from get_data import Data

data = Data()
data.print_message()

from fileName import className
